In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [45]:
url_inicial = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2015'
url_root = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2015'
r=requests.get(url_inicial)

In [46]:
soup = BeautifulSoup(r.text, 'html.parser')

In [47]:
box = soup.find('div', class_='sumarios decada-2015')
volumen=box.findAll('div', class_='sumario row')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-4-S0187533716X00027',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-3-S0187533715X00045',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-2-S0187533715X00033',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-1-S0187533715X00021']

In [48]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='items') 
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1

In [49]:
vol2

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-evaluacion-pacientes-con-problemas-fertilidad-S0187533718300591',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-relationship-between-follicular-fluid-serum-S0187533718300062',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-relacion-entre-factores-psicosociales-el-S0187533718300372',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-obstetrician39s-attitudes-about-delivery-through-S0187533718300359',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-recien-nacidos-vivos-con-defectos-S0187533718300347',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-manejo-del-dolor-el-recien-S0187533718300566',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-utilidad-biomarcadores-sericos-involucrados-fisiopa

In [50]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [51]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2015


In [52]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

32

In [53]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [54]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perinatología y Reproducción Humana'
    a='III. Medicina y Ciencias de la Salud'
    tem='Perinatología, Neonatología, Obstetricia, Reproducción Humana'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('span', class_='elsevierStyleSections').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('div', class_='right').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [55]:
list_scraper=list_scraper[0:32]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31


In [56]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Evaluación de las pacientes con problemas de f...,En el presente número de Perinatología y Repro...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
1,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Relationship between follicular fluid and seru...,Summary of relevanceGeneral criticisms:Few stu...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
2,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Relación entre factores psicosociales y el Índ...,IntroducciónSegún la Organización Mundial de l...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
3,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Obstetrician's attitudes about delivery throug...,IntroductionCesarean section is the most commo...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
4,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Recién nacidos vivos con defectos del tubo neu...,IntroducciónLos defectos del cierre del tubo n...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
5,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Manejo del dolor en el recién nacido postopera...,IntroducciónEl dolor se ha descrito como un pr...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
6,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Utilidad de los biomarcadores séricos involucr...,IntroducciónLa preeclampsia (PE) es la causa m...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
7,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...","Tamizaje neonatal de hipotiroidismo congénito,...",Antecedentes históricosEl objetivo de los prog...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
8,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Evaluación de las pacientes con problemas de f...,En el presente número de Perinatología y Repro...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
9,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Relationship between follicular fluid and seru...,Summary of relevanceGeneral criticisms:Few stu...,None,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus


In [57]:
df_catalogo.to_csv('Revista95.04.csv', index=False)